In [1]:
# Исследовать структуру html-файлов, чтобы произвести парсинг всех данных. 
# В каждом файле содержится информация об одном объекте из случайной предметной области. 
# Полученные данные собрать и записать в json. 
# Выполните также ряд операций с данными:
# 1) отсортируйте значения по одному из доступных полей
# 2) выполните фильтрацию по другому полю (запишите результат отдельно)
# 3) для одного выбранного числового поля посчитайте статистические характеристики (сумма, мин/макс, среднее и т.д.)
# 4) для одного текстового поля посчитайте частоту меток

In [1]:
from bs4 import BeautifulSoup
import re
import json
import math
import collections
import pandas as pd

In [2]:
def handle_file(file_name):
    with open(file_name, encoding = 'utf-8') as file:
        text = ""
        for row in file.readlines():
            text += row
        item = dict()
        site = BeautifulSoup(text, 'html.parser')
        
        # print(site.prettify())
        item['category'] = site.find_all("span", string = re.compile('Категория:'))[0].get_text().split(':')[1].strip()
        item['title'] = site.find_all('h1')[0].get_text().strip()
        item['author'] = site.find_all("p", attrs = {'class' : 'author-p'})[0].get_text().strip()
        item['pages'] = int(site.find_all("span", attrs = {'class' : 'pages'})[0].get_text().split(':')[1].replace(" страниц", "").strip())
        item['year'] = int(site.find_all('span', attrs = {'class': 'year'})[0].get_text().replace("Издано в ", "").strip())
        item['ISBN'] = site.find_all('span', string = re.compile('ISBN'))[0].get_text(). split(':')[1].strip()
        item['description'] = site.find_all("p")[1].get_text().replace("Описание \n", "").strip()
        item['img'] = site.find_all('img')[0]['src']
        item['rate'] = float(site.find_all('span', string = re.compile('Рейтинг:'))[0].get_text().split(':')[1].strip())
        item['views'] = int(site.find_all('span', string = re.compile('Просмотры:'))[0].get_text().split(':')[1].strip())
        return item

In [3]:
items = []
for i in range(1,1000):
  file_name = f'./1/{i}.html'
  result = handle_file(file_name)
  items.append(result)
print(items[1:2])

[{'category': 'фэнтези', 'title': 'Великий Гэтсби', 'author': 'Джозеф Конрад', 'pages': 108, 'year': 1869, 'ISBN': '792-5260761137', 'description': 'Описание\n     middle', 'img': 'https://upload.freebook.xyz/718947.jpeg', 'rate': 4.5, 'views': 38845}]


In [4]:
with open('result_all_63_1.json', 'w', encoding = 'utf-8') as f:
    f.write(json.dumps(items, ensure_ascii=False))

Отсортируeм значения по полю ['year'] в убывающем порядке:

In [5]:
items = sorted(items, key = lambda x: x['year'], reverse = True)
print(items[1:5])

[{'category': 'исторический роман', 'title': 'Лев, колдунья и платяной шкаф', 'author': 'Тед Хьюз', 'pages': 383, 'year': 2023, 'ISBN': '482-7986769437', 'description': 'Описание\n     write it yourself', 'img': 'https://upload.booook.ru/273008.jpeg', 'rate': 3.3, 'views': 30128}, {'category': 'фэнтези', 'title': 'Портрет Дориана Грея', 'author': 'Патрик Зюскинд', 'pages': 322, 'year': 2022, 'ISBN': '778-8939592605', 'description': 'Описаниеsmall', 'img': 'https://upload.freebook.xyz/95659.png', 'rate': 2.8, 'views': 48880}, {'category': 'приключения', 'title': '20 000 лье под водой', 'author': 'Станислав Лем', 'pages': 121, 'year': 2022, 'ISBN': '598-3645209478', 'description': 'Описаниеwrite it yourself', 'img': 'https://upload.abc-book.it/604859.jpg', 'rate': 2.0, 'views': 27691}, {'category': 'детская литература', 'title': 'О дивный новый мир', 'author': 'Марк Твен', 'pages': 241, 'year': 2022, 'ISBN': '595-6864703051', 'description': 'Описаниеmiddle', 'img': 'https://upload.abc-bo

Выполним фильтрацию по полю ['author'] = 'Фрэнсис Скотт Фицджеральд':

In [6]:
filtered_items = []
for book in items:
    if book['author'] == 'Фрэнсис Скотт Фицджеральд':
        filtered_items.append(book)
print(filtered_items[1:6])        

[{'category': 'роман', 'title': 'Заводной апельсин', 'author': 'Фрэнсис Скотт Фицджеральд', 'pages': 204, 'year': 1981, 'ISBN': '720-5028495410', 'description': 'Описаниеvery impressive', 'img': 'https://upload.book.com/280850.jpg', 'rate': 1.0, 'views': 81487}, {'category': 'научная фантастика', 'title': 'Американские горки', 'author': 'Фрэнсис Скотт Фицджеральд', 'pages': 337, 'year': 1970, 'ISBN': '385-9948804752', 'description': 'Описание\n     description', 'img': 'https://upload.book.com/570367.jpg', 'rate': 1.3, 'views': 20649}, {'category': 'приключения', 'title': 'Трудно быть богом', 'author': 'Фрэнсис Скотт Фицджеральд', 'pages': 371, 'year': 1964, 'ISBN': '168-9608476516', 'description': 'Описаниеmiddle', 'img': 'https://upload.buybook.org/208343.jpeg', 'rate': 5.0, 'views': 35271}, {'category': 'фэнтези', 'title': 'Звездная пыль', 'author': 'Фрэнсис Скотт Фицджеральд', 'pages': 449, 'year': 1959, 'ISBN': '252-4293258910', 'description': 'Описаниеdescription', 'img': 'https:

In [7]:
with open("result_filtered_63_1.json", "w", encoding = 'utf-8') as file:
    file.write(json.dumps(filtered_items, ensure_ascii=False))

для числового поля ['pages'] посчитаем статистические характеристики (сумма, мин/макс, среднее и т.д.)

In [8]:
stats_items = []
df = pd.DataFrame(items)
pd.set_option('display.float_format', '{:.1f}'.format)
stats = df['pages'].agg(['max', 'min', 'mean', 'median', 'std']).to_dict()
stats_items.append(stats)
print(stats_items)

[{'max': 500.0, 'min': 100.0, 'mean': 300.2762762762763, 'median': 303.0, 'std': 115.63655725806075}]


для текстового поля ['title'] посчитайте частоту меток

In [9]:
text_items = []
text = [item['title'] for item in items]
f1 = collections.Counter(text)
text_items.append(f1)
print(text_items)

[Counter({'Гроздья гнева': 22, 'Женщина в песках': 20, 'Скотный двор': 20, 'Сумерки': 18, 'Черный дом': 18, 'Ярмарка тщеславия': 17, 'Бойня номер пять, или Крестовый поход детей': 17, 'Плавучий трава': 17, 'Зеленая миля': 16, 'Бесконечная история': 16, 'Франкенштейн': 16, 'Процесс': 15, 'Идиот': 15, 'Лев, колдунья и платяной шкаф': 14, 'Солярис': 14, 'Слепота': 14, 'Твин Пикс: Воспоминания специального агента ФБР Дейла Купера': 14, 'Прощай, оружие': 14, 'Котлован': 14, 'Алхимик': 14, 'Портрет Дориана Грея': 13, 'Трудно быть богом': 13, 'Повелитель мух': 13, 'Сто лет одиночества': 13, 'Граф Монте-Кристо': 13, 'Эпоха невинности': 13, 'Звездная пыль': 13, 'Страх и отвращение в Лас-Вегасе': 13, 'Ребекка': 13, 'О дивный новый мир': 12, 'Волхв': 12, '451 градус по Фаренгейту': 12, 'Под стеклянным колпаком': 12, 'Гордость и предубеждение': 12, 'По направлению к сваолу': 12, 'Хроники Нарнии': 12, 'Гарри Поттер и Философский камень': 12, 'Имя розы': 12, 'Оно': 12, 'Убить пересмешника': 12, 'Вел